# NASA Breath Diagnostics Challenge

## Exploratory Data Analysis

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [2]:
STAGING_DATA_FOLDER = "staging_data"
FIGURES_FOLDER = "figures"

In [3]:
train_readings_df = pd.read_csv(os.path.join(STAGING_DATA_FOLDER, "train_readings.csv"))
train_patients_df = pd.read_csv(os.path.join(STAGING_DATA_FOLDER, "train_patients.csv"))

In [4]:
train_readings_df.describe()

,Time,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D56,D57,D58,D59,D60,D61,D62,D63,D64,Patient ID
count,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,...,1.701100e+04,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000,17011.000000
mean,427.756446,82.790582,13.124385,57.181407,35.299644,802.625789,195.202339,909.319536,106580.189778,1231.704583,...,9.222690e+06,13.302048,47.422683,9236.279933,542.823495,388.535925,103.033607,798.463564,5166.451811,32.127153
std,248.207513,77.235479,15.444895,71.305168,32.477778,1141.516795,329.476614,1200.539756,135908.980557,2892.254308,...,5.261053e+06,10.515801,23.220846,7204.872971,447.665081,359.089099,76.080721,562.985053,3969.979817,18.371027
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,213.218000,27.189450,2.541590,5.319580,11.227497,174.612450,16.042655,82.456140,12581.730891,44.008233,...,5.167139e+06,6.065380,31.669430,4060.494500,193.995779,182.323405,51.498560,386.142708,2187.000263,17.000000
50%,427.764000,57.661600,6.195900,35.108214,23.624939,404.241220,68.977056,282.456140,39218.589083,274.314020,...,8.222922e+06,10.537928,44.247790,7612.880644,446.789400,282.833940,83.820562,692.618483,4290.517600,31.000000
75%,641.007500,116.556596,18.535653,86.895856,49.747858,918.038115,156.529194,1403.508772,169065.441296,1550.083789,...,1.297690e+07,17.078090,59.004910,12323.462647,750.488450,471.592730,129.935940,1076.689984,7015.815920,46.000000
max,1023.768000,526.115979,85.543796,570.216641,164.560731,12619.734520,1881.399120,6577.192980,703844.107475,29715.009500,...,2.715138e+07,68.051648,158.117577,41594.890158,2804.609520,3124.089720,562.363240,4316.679800,29731.386000,63.000000


In [5]:
train_patients_df.describe()

,Patient ID,Result
count,45.000000,45.000000
mean,32.088889,0.422222
std,18.584967,0.499495
min,1.000000,0.000000
25%,17.000000,0.000000
50%,31.000000,0.000000
75%,46.000000,1.000000
max,63.000000,1.000000


In [6]:
train_readings_df.groupby("Patient ID").size().reset_index()

,Patient ID,0
0,1,376
1,4,375
2,5,374
3,6,375
4,7,376
5,8,375
6,9,375
7,10,375
8,12,378
9,13,375


In [7]:
train_readings_df[~train_readings_df["Patient ID"].isin([20, 53])].describe()

,Time,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D56,D57,D58,D59,D60,D61,D62,D63,D64,Patient ID
count,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,...,1.612200e+04,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000,16122.000000
mean,423.634298,83.159477,13.254945,57.347048,35.407632,803.647125,202.483191,878.658642,109465.280520,1291.092432,...,9.255941e+06,13.277270,45.731902,9334.741786,549.669223,391.228643,99.651782,784.270037,5094.336179,31.868627
std,244.984523,78.557044,15.624877,72.319728,32.699583,1153.758812,336.627391,1184.129030,137881.560219,2959.298610,...,5.337363e+06,10.679842,20.359880,7326.551454,454.648810,363.602966,72.721932,561.903171,3931.473404,18.434183
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,211.135500,26.805949,2.516252,5.361930,11.265638,175.210010,17.149866,76.718363,12833.366754,47.576629,...,5.132765e+06,5.970739,31.252271,4051.863857,195.145576,184.155504,50.310516,377.216548,2139.964956,16.000000
50%,423.700000,56.883950,6.219239,35.368488,23.664087,400.143067,69.680103,266.666667,39435.579200,304.446748,...,8.149816e+06,10.397514,43.644299,7599.165984,448.418080,284.683897,81.809063,675.589547,4243.281486,31.000000
75%,636.164500,116.972888,19.048772,85.941682,50.108201,905.319317,166.069285,1317.105263,177320.387337,1616.942844,...,1.319125e+07,16.779954,58.066425,12471.958350,761.391171,471.735952,126.366520,1053.854869,6943.753100,46.000000
max,853.800000,526.115979,85.543796,570.216641,164.560731,12619.734520,1881.399120,6577.192980,703844.107475,29715.009500,...,2.715138e+07,68.051648,143.396226,41594.890158,2804.609520,3124.089720,562.363240,4316.679800,29731.386000,63.000000


In [8]:
train_readings_df = train_readings_df[train_readings_df["Time"] < 841.0]
train_readings_df.describe()

,Time,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D56,D57,D58,D59,D60,D61,D62,D63,D64,Patient ID
count,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,...,1.674000e+04,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000,16740.000000
mean,420.310704,82.860539,13.259755,56.904035,35.460538,805.335339,196.385323,904.591992,107313.631314,1238.936845,...,9.199025e+06,13.391270,47.358463,9237.669572,544.526376,390.793880,103.662284,807.240630,5234.649009,32.088889
std,243.059342,77.242866,15.513423,71.369837,32.652540,1147.399536,331.258830,1201.870282,136462.549715,2906.245847,...,5.266091e+06,10.520823,22.965330,7216.660528,448.589646,360.465907,76.270578,562.516029,3964.028493,18.377856
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,210.151000,27.385983,2.572263,5.205081,11.234849,175.454646,15.965844,80.701760,12567.763892,44.978341,...,5.142722e+06,6.158325,31.797070,4058.225346,194.494287,183.798078,51.967751,395.151308,2251.428698,17.000000
50%,420.352500,57.655560,6.301700,34.525775,23.574080,404.384635,68.936165,278.947368,39440.758819,273.523004,...,8.188835e+06,10.645804,44.300747,7596.778900,447.067481,285.122986,84.478865,703.301520,4362.368100,31.000000
75%,630.503250,116.579295,18.992491,86.298744,50.740418,915.551335,157.919620,1385.964912,171701.374340,1557.278401,...,1.294530e+07,17.132429,58.949630,12317.460400,754.197794,474.686683,130.535930,1083.348989,7068.428403,46.000000
max,840.667000,526.115979,85.543796,570.216641,164.560731,12619.734520,1881.399120,6577.192980,703844.107475,29715.009500,...,2.715138e+07,68.051648,158.117577,41594.890158,2804.609520,3124.089720,562.363240,4316.679800,29731.386000,63.000000


In [9]:
train_readings_df.groupby("Patient ID").size().reset_index()

,Patient ID,0
0,1,372
1,4,372
2,5,372
3,6,372
4,7,372
5,8,372
6,9,372
7,10,372
8,12,372
9,13,372


In [10]:
# fig, ax = plt.subplots(2, 2, figsize=(16, 12))
# ax[0][0].set_title("Bin Width = 0.5")
# _ = sns.histplot(train_readings_df, x="Time", binwidth=0.5, kde=True, ax=ax[0][0])
# ax[0][1].set_title("Bin Width = 1")
# _ = sns.histplot(train_readings_df, x="Time", binwidth=1, kde=True, ax=ax[0][1])
# ax[1][0].set_title("Bin Width = 2")
# _ = sns.histplot(train_readings_df, x="Time", binwidth=2, kde=True, ax=ax[1][0])
# ax[1][1].set_title("Bin Width = 2.26")
# _ = sns.histplot(train_readings_df, x="Time", binwidth=2.26, kde=True, ax=ax[1][1])
# fig.savefig(os.path.join(FIGURES_FOLDER, "time_histograms.png"), bbox_inches="tight")
# plt.close()

![Histograms for Time column](figures/time_histograms.png)

In [11]:
PATIENT_IDS = [1, 4, 10, 12]
# 1 and 12 have COVID, 4 and 10 don't have COVID

In [12]:
sample_df = train_readings_df[train_readings_df["Patient ID"].isin(PATIENT_IDS)]
# sample_df = sample_df.drop(columns=[x for x in sample_df.columns if x == "Patient ID"])
sample_df

,Time,D1,D2,D3,D4,D5,D6,D7,D8,D9,...,D56,D57,D58,D59,D60,D61,D62,D63,D64,Patient ID
1500,0.310,44.953990,6.016480,4.424111,16.668637,886.555847,11.909577,428.070175,2905.811623,15.571704,...,3.994169e+06,14.526012,52.000578,6176.841347,271.422809,427.191487,166.420295,1016.607644,8176.257646,10
1501,2.592,47.669332,6.670445,4.457627,17.811404,869.058034,12.358995,487.719298,3295.479849,16.758120,...,8.543732e+06,15.495990,57.417305,6356.924185,271.422809,401.363707,175.003576,1068.506533,8904.423085,10
1502,4.842,49.479560,6.496054,4.507901,18.599519,1093.613298,12.628646,564.912281,3579.380984,20.296180,...,6.497085e+06,14.336748,42.611585,6221.862057,267.388924,480.396715,172.619331,1041.030651,8342.695460,10
1503,7.125,46.764218,4.621354,4.189499,12.334003,971.128609,10.651207,417.543860,2655.310621,16.991165,...,3.543732e+06,14.100168,41.889354,6212.857915,269.694001,462.317268,161.651805,976.920259,8176.257646,10
1504,9.377,45.859104,4.316170,4.055435,10.757773,487.022456,9.932138,380.701754,2321.309285,15.592890,...,7.842566e+06,13.910904,53.083923,6221.862057,266.812655,404.463040,165.943446,967.761631,8113.843465,10
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12881,831.530,117.848792,14.274744,60.277323,27.570603,640.133867,662.761425,149.326939,166654.489662,3670.762752,...,1.605889e+07,12.634656,41.644034,15401.827335,386.083744,172.800937,4.753755,123.977188,1799.322608,4
12882,833.776,110.989867,14.590956,60.001183,27.151596,704.442549,657.920217,148.600287,165777.745306,3669.191364,...,1.230035e+07,12.728944,46.351620,15196.718255,382.389163,239.675876,25.194904,195.264071,1862.828112,4
12883,836.026,115.042868,14.274744,59.744768,32.389173,641.446289,655.983734,147.873635,164827.938920,3661.020145,...,1.416843e+07,12.469651,37.298570,15243.333955,377.463054,166.455140,32.800913,123.977188,1820.491109,4
12884,838.310,121.278254,14.184397,59.567250,25.978379,595.183411,655.015492,147.691972,165120.187039,3657.248814,...,1.482686e+07,12.776088,25.710664,15094.163714,379.310345,218.685932,20.916524,3.099430,1608.806097,4


In [13]:
# fig, ax = plt.subplots(8, 8, figsize=(64, 64))
# for i in range(64):
#     _ = sns.lineplot(sample_df, x="Time", y=f"D{i+1}", hue="Patient ID", palette="tab10", ax=ax[i // 8][i % 8])
# fig.savefig(os.path.join(FIGURES_FOLDER, "raw_samples_timeline.png"), bbox_inches="tight")
# plt.close()

![Raw Samples on Timeline](figures/raw_samples_timeline.png)

In [14]:
patient_means = sample_df[sample_df["Time"] < 301].groupby("Patient ID").mean().reset_index().drop(columns="Time")
result = sample_df.merge(patient_means, on="Patient ID", suffixes=("_sample", "_mean"))
result

,Time,D1_sample,D2_sample,D3_sample,D4_sample,D5_sample,D6_sample,D7_sample,D8_sample,D9_sample,...,D55_mean,D56_mean,D57_mean,D58_mean,D59_mean,D60_mean,D61_mean,D62_mean,D63_mean,D64_mean
0,0.310,44.953990,6.016480,4.424111,16.668637,886.555847,11.909577,428.070175,2905.811623,15.571704,...,1118.975488,6.051207e+06,8.691914,54.924796,3163.906525,134.847001,376.747696,122.715114,748.757962,6664.55178
1,2.592,47.669332,6.670445,4.457627,17.811404,869.058034,12.358995,487.719298,3295.479849,16.758120,...,1118.975488,6.051207e+06,8.691914,54.924796,3163.906525,134.847001,376.747696,122.715114,748.757962,6664.55178
2,4.842,49.479560,6.496054,4.507901,18.599519,1093.613298,12.628646,564.912281,3579.380984,20.296180,...,1118.975488,6.051207e+06,8.691914,54.924796,3163.906525,134.847001,376.747696,122.715114,748.757962,6664.55178
3,7.125,46.764218,4.621354,4.189499,12.334003,971.128609,10.651207,417.543860,2655.310621,16.991165,...,1118.975488,6.051207e+06,8.691914,54.924796,3163.906525,134.847001,376.747696,122.715114,748.757962,6664.55178
4,9.377,45.859104,4.316170,4.055435,10.757773,487.022456,9.932138,380.701754,2321.309285,15.592890,...,1118.975488,6.051207e+06,8.691914,54.924796,3163.906525,134.847001,376.747696,122.715114,748.757962,6664.55178
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,831.530,117.848792,14.274744,60.277323,27.570603,640.133867,662.761425,149.326939,166654.489662,3670.762752,...,3362.990643,5.343110e+06,11.626549,43.906616,5368.025695,103.517723,222.535985,62.066890,419.634817,2337.12987
1484,833.776,110.989867,14.590956,60.001183,27.151596,704.442549,657.920217,148.600287,165777.745306,3669.191364,...,3362.990643,5.343110e+06,11.626549,43.906616,5368.025695,103.517723,222.535985,62.066890,419.634817,2337.12987
1485,836.026,115.042868,14.274744,59.744768,32.389173,641.446289,655.983734,147.873635,164827.938920,3661.020145,...,3362.990643,5.343110e+06,11.626549,43.906616,5368.025695,103.517723,222.535985,62.066890,419.634817,2337.12987
1486,838.310,121.278254,14.184397,59.567250,25.978379,595.183411,655.015492,147.691972,165120.187039,3657.248814,...,3362.990643,5.343110e+06,11.626549,43.906616,5368.025695,103.517723,222.535985,62.066890,419.634817,2337.12987


In [15]:
for col in [f"D{i+1}" for i in range(64)]:
    result[col] = result[f"{col}_sample"] - result[f"{col}_mean"]
    result.drop(columns=[f"{col}_sample", f"{col}_mean"], inplace=True)

In [16]:
result

,Time,Patient ID,D1,D2,D3,D4,D5,D6,D7,D8,...,D55,D56,D57,D58,D59,D60,D61,D62,D63,D64
0,0.310,10,24.508397,4.401727,2.480940,11.944390,185.393492,7.614763,270.650310,2077.588761,...,338.740973,-2.057038e+06,5.834098,-2.924218,3012.934822,136.575808,50.443791,43.705181,267.849683,1511.705866
1,2.592,10,27.223739,5.055692,2.514456,13.087157,167.895680,8.064181,330.299433,2467.256987,...,764.992884,2.492525e+06,6.804076,2.492509,3193.017660,136.575808,24.616011,52.288463,319.748572,2239.871305
2,4.842,10,29.033967,4.881301,2.564730,13.875272,392.450944,8.333832,407.492415,2751.158122,...,761.739053,4.458778e+05,5.644834,-12.313211,3057.955532,132.541924,103.649019,49.904218,292.272689,1678.143680
3,7.125,10,26.318625,3.006601,2.246328,7.609756,269.966254,6.356393,260.123994,1827.087759,...,647.854954,-2.507475e+06,5.408254,-13.035441,3048.951390,134.847001,85.569572,38.936691,228.162297,1511.705866
4,9.377,10,25.413511,2.701417,2.112264,6.033526,-214.139899,5.637324,223.281889,1493.086423,...,869.115489,1.791359e+06,5.218990,-1.840873,3057.955532,131.965654,27.715344,43.228332,219.003670,1449.291685
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1483,831.530,4,96.775063,12.032045,55.281748,22.608058,422.871274,603.276442,131.213914,150515.471846,...,1435.090124,1.071578e+07,1.008107,-2.262582,10033.801640,282.566021,-49.735047,-57.313135,-295.657629,-537.807262
1484,833.776,4,89.916139,12.348258,55.005608,22.189052,487.179955,598.435234,130.487262,149638.727490,...,895.306038,6.957243e+06,1.102395,2.445004,9828.692559,278.871440,17.139892,-36.871986,-224.370746,-474.301758
1485,836.026,4,93.969140,12.032045,54.749193,27.426628,424.183696,596.498751,129.760611,148688.921105,...,1361.786113,8.825323e+06,0.843102,-6.608046,9875.308259,273.945331,-56.080845,-29.265977,-295.657629,-516.638760
1486,838.310,4,100.204525,11.941699,54.571674,21.015835,377.920818,595.530509,129.578948,148981.169223,...,1231.838092,9.483745e+06,1.149540,-18.195952,9726.138019,275.792622,-3.850053,-41.150366,-416.535388,-728.323773


In [17]:
# fig, ax = plt.subplots(8, 8, figsize=(64, 64))
# for i in range(64):
#     col = f"D{i+1}"
#     _ = sns.lineplot(result, x="Time", y=col, hue="Patient ID", palette="tab10", ax=ax[i // 8][i % 8])
# fig.savefig(os.path.join(FIGURES_FOLDER, "raw_samples_timeline_trend_removed.png"), bbox_inches="tight")
# plt.close()

![Raw Samples on Timeline with Trend Removed](figures/raw_samples_timeline_trend_removed.png)

In [18]:
fig, ax = plt.subplots(1, 1, figsize=(64, 64))
corr = result[[f"D{i+1}" for i in range(64)]].corr()
_ = sns.heatmap(
    corr,
    vmin=-1,
    vmax=1,
    center=0,
    cmap="seismic",
    cbar=False,
    annot=True,
    fmt=".2f",
    square=True,
    ax=ax,
)
fig.savefig(os.path.join(FIGURES_FOLDER, "feature_heatmap.png"), bbox_inches="tight")
plt.close()

![Heatmap of Feature Correlations](figures/feature_heatmap.png)

In [19]:
fig, ax = plt.subplots(1, 1, figsize=(64, 64))
corr = result[[f"D{i+1}" for i in range(64)]].corr()
corr = corr.map(lambda x: x if abs(x) >= 0.95 else 0.0)
_ = sns.heatmap(
    corr,
    vmin=-1,
    vmax=1,
    center=0,
    cmap="seismic",
    cbar=False,
    annot=True,
    fmt=".2f",
    square=True,
    ax=ax,
)
fig.savefig(os.path.join(FIGURES_FOLDER, "feature_heatmap_masked95.png"), bbox_inches="tight")
plt.close()

![Feature Correlation Heatmap masked for high correlation](figures/feature_heatmap_masked95.png)